In [89]:
import PyPDF2
import re
import pandas as pd
import yfinance as yf

# TODO API yahoo finance

pdf_file = open('ru2000_membershiplist_20220624_0.pdf', 'rb')
read_pdf = PyPDF2.PdfReader(pdf_file)
pages = read_pdf.pages
p = 0
#df = pd.DataFrame(columns = ['Company', 'Ticker', 'IPO', 'First Value', 'Last Value', 'Last Date', 'How Much'])


In [94]:
co = yf.Ticker("GOOG")
history = co.history(period="max", interval="3mo",)
print(history.index[0])
print(history.index[-1])


2004-08-01 00:00:00-04:00
2023-05-01 00:00:00-04:00


In [98]:
co = yf.Ticker("GOOG")
history = co.history(period="max")
print(history.index[0])
print(history.index[-1])


2004-08-19 00:00:00-04:00
2023-05-05 00:00:00-04:00


In [99]:

from datetime import datetime, timezone
 
# Get the datetime object using datetime
# module
dt_obj_w_tz = datetime.now()
print(dt_obj_w_tz)
 
# Add timezone information to the datetime
# object
dt_obj_w_tz = dt_obj_w_tz.replace(tzinfo=timezone.utc)
print(dt_obj_w_tz)
 
# Remove the timezone information from the datetime
# object
dt_obj_wo_tz = dt_obj_w_tz.replace(tzinfo=None)
print(dt_obj_wo_tz)

2023-05-07 19:22:16.794701
2023-05-07 19:22:16.794701+00:00
2023-05-07 19:22:16.794701


In [100]:
type(history)
row = history.iloc[0]
date = history.index[0]
type(date)
print(date)
print(date.replace(tzinfo=None))
#history.dtypes
history.to_csv("teste.csv", date_format='%Y-%m-%d')

2004-08-19 00:00:00-04:00
2004-08-19 00:00:00


In [104]:
def addRow(rows, company, ticker):
	co = yf.Ticker(ticker)
	history = co.history(period="max", interval="3mo")
	try:
		ipo = history.index[0].tz_localize(None)
		firstValue = history.iloc[0,0]
		lastValue = history.iloc[-1,0]
		lastDate = history.index[-1].tz_localize(None)
		howMuchPercent = ((lastValue - firstValue) / firstValue) * 100
	except:
		ipo = None
		firstValue = None
		lastValue = None
		lastDate = None
		howMuchPercent = None
	row = [company, tickeripo, firstValue, lastValue, lastDate, howMuchPercent]
	# add row in the dataframe
	rows.append(row)
	return 


columns = ['Company', 'Ticker', 'IPO', 'First Value', 'Last Value',
	   		 'Last Date', 'How Much']
rows = []
# Lê até encontrar as strings que indicam final
while True:
	page = pages[p]
	page_content = page.extract_text()
	parsed = ''.join(page_content)
	splited = parsed.split('\n')
	print("\n\n---------------------")
	print("Página " + str(p+1))
	print("---------------------\n")
	if p != 9: # Tratamento normal - a não ser a página 10 (no else) que está fora do padrão
		c = 0
		#cabeçalhos
		print(splited[c] + " | " + splited[c+1])
		c += 2
		ended = False
		while splited[c] != "June 24, 2022": #String do fim de cada página
			if splited[c] == "ftserussell.com": #String no final de toda a lista
				print("----------------\nTerminou\n----------------")
				ended = True
				break
			company = splited[c]
			ticker = splited[c+1]
			addRow(rows, company, ticker)
			#df.loc[len(df.index)] = [company, ticker, ipo, firstValue, lastValue, lastDate, howMuchPercent]
			print( company + " | " + ticker + " | ̭̭" + str(ipo) + " | ̭̭" + str(firstValue) + " | ̭̭" + str(lastValue) + " | ̭̭" + str(lastDate) + " | ̭̭" + str(howMuchPercent))
			c += 2
			#df.to_csv('SuperTempShares.csv', date_format="%d/%m/%Y")
			df = pd.DataFrame(rows, columns = columns)
			print(df)
			df.to_excel('SuperTempShares.xlsx')


	else:
		# Na página 10 o PDF fica sem quebra de linhas entre colunas
		
		# cabeçalhos
		c = 0
		# separa o ticker, que está no lado direito
		headers = splited[c].rsplit(maxsplit=1)
		print(headers[0] + " | " + headers[1])
		c += 1
		while splited[c][:13] != "June 24, 2022":
			# separa o ticker, que está a direita
			subsplited = splited[c].rsplit(maxsplit=1)
			company = subsplited[0]
			ticker = subsplited[1]
			addRow(rows, ticker)
			#df.loc[len(df.index)] = [company, ticker, ipo, firstValue, lastValue, lastDate, howMuchPercent]
			print( company + " | " + ticker + " | ̭̭" + str(ipo) + " | ̭̭" + str(firstValue) + " | ̭̭" + str(lastValue) + " | ̭̭" + str(lastDate) + " | ̭̭" + str(howMuchPercent))
			c += 1

	if ended:
		print("----------------\nTerminou Mesmo\n----------------")
		break

	p += 1
	df = pd.DataFrame(rows, columns = columns)
	
	#print(df)
	#df.to_excel('TempShares.xlsx')

print(df)
df.to_excel('Shares.xlsx')

#print("Sem eliminar as quebras")
#print(parsed)

#parsed = re.sub('n', '', parsed)
#print("Após eliminar as quebras")
#print(parsed)

#print("Pegando apenas as 20 primeiras posições")
#novastring = parsed[0:20]
#print(novastring)




---------------------
Página 3
---------------------

Company | Ticker
APELLIS PHARMACEUTICALS | APLS | ̭̭2017-11-01 00:00:00-04:00 | ̭̭13.869999885559082 | ̭̭76.70999908447266 | ̭̭2023-05-01 00:00:00-04:00 | ̭̭453.06416523002423


ValueError: 7 columns passed, passed data had 5 columns